In [ ]:
import pandas as pd
import ast

import numpy as np

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

True

In [4]:
df = pd.read_csv('./final_cocktails.csv')

df = df[df.columns[2:]]

In [5]:
df.head(5)

,name,alcoholic,category,glassType,instructions,drinkThumbnail,ingredients,ingredientMeasures,text
0,A1,Alcoholic,Cocktail,Cocktail glass,"Pour all ingredients into a cocktail shaker, m...",https://www.thecocktaildb.com/images/media/dri...,"['Gin', 'Grand Marnier', 'Lemon Juice', 'Grena...","['1 3/4 shot ', '1 Shot ', '1/4 Shot', '1/8 Sh...",question Generate a cocktail with Gin Grand Ma...
1,ABC,Alcoholic,Shot,Shot glass,Layered in a shot glass.,https://www.thecocktaildb.com/images/media/dri...,"['Amaretto', 'Baileys irish cream', 'Cognac']","['1/3 ', '1/3 ', '1/3 ']",question Generate a cocktail with Amaretto Bai...
2,Ace,Alcoholic,Cocktail,Martini Glass,Shake all the ingredients in a cocktail shaker...,https://www.thecocktaildb.com/images/media/dri...,"['Gin', 'Grenadine', 'Heavy cream', 'Milk', 'E...","['2 shots ', '1/2 shot ', '1/2 shot ', '1/2 sh...",question Generate a cocktail with Gin Grenadin...
3,ACID,Alcoholic,Shot,Shot glass,Poor in the 151 first followed by the 101 serv...,https://www.thecocktaildb.com/images/media/dri...,"['151 proof rum', 'Wild Turkey']","['1 oz Bacardi ', '1 oz ']",question Generate a cocktail with 151 proof ru...
4,Adam,Alcoholic,Ordinary Drink,Cocktail glass,"In a shaker half-filled with ice cubes, combin...",https://www.thecocktaildb.com/images/media/dri...,"['Dark rum', 'Lemon juice', 'Grenadine']","['2 oz ', '1 oz ', '1 tsp ']",question Generate a cocktail with Dark rum Lem...


In [6]:
def clean_ingredients(row):
    try:
        ingredients = ast.literal_eval(row['ingredients']) if isinstance(row['ingredients'], str) else row['ingredients']
        measures = ast.literal_eval(row['ingredientMeasures']) if isinstance(row['ingredientMeasures'], str) else row['ingredientMeasures']
        
        if isinstance(ingredients, list) and isinstance(measures, list):
            return [f"{m} {i}" if m else i for i, m in zip(ingredients, measures)]
        
        return ingredients
    except:
        return []

df['general_ingredients'] = df.apply(clean_ingredients, axis=1)

In [7]:
def generate_text(row):
    ingredients_text = ", ".join(row["general_ingredients"]) if isinstance(row["general_ingredients"], list) else ""
    
    return f"{row['name']} is a {row['category']} cocktail served in a {row['glassType']}. " \
           f"It is {'alcoholic' if row['alcoholic'].lower() == 'alcoholic' else 'non-alcoholic'}. " \
           f"Ingredients: {ingredients_text}. Instructions: {row['instructions']}."

df['text'] = df.apply(generate_text, axis=1)

In [8]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

C:\Users\TaharMasmaliyev\AppData\Local\Temp\ipykernel_17916\412152783.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\TaharMasmaliyev\.conda\envs\RagPy3.10\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
df["embedding"] = df["text"].apply(lambda x: embedding_model.embed_query(x) if pd.notna(x) else np.zeros(384))

In [ ]:
df['embedding']

In [10]:
embeddings = np.vstack(df["embedding"].values)

In [ ]:
vector_db = FAISS.from_texts(
    texts=df["text"].tolist(),
    embedding=embedding_model
)

In [15]:
vector_db.save_local("faiss_cocktails")